In [100]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
import xgboost
import lightgbm as lgb
from category_encoders import TargetEncoder

In [101]:
# 读取数据
data = pd.read_csv('train.csv')
data_target = data['fraud']
data = data.drop(data.columns[-1], axis=1)


In [102]:
# 与平均年龄的差值
age_mean = data['age'].mean()
data['age_diff_mean'] = np.abs(data['age'] - age_mean)

# 将'customer_months'列的值取负
data['customer_months_miunus'] = -data['customer_months']


# 将'policy_state'和'insured_sex'列转换为独热编码
data = pd.get_dummies(data, columns=['policy_state', 'insured_sex'])
# 构造policy_csl_total特征
def get_csl_total(csl_str):
    csl_min, csl_max = map(int, csl_str.split('/'))
    return csl_min + csl_max
data['policy_csl_total'] = data['policy_csl'].apply(get_csl_total)


# 构造policy_csl_level特征
def get_csl_level(csl_str):
    csl_min, csl_max = map(int, csl_str.split('/'))
    csl_total = csl_min + csl_max
    if csl_total < 500:
        return 'low'
    elif csl_total < 1000:
        return 'medium'
    else:
        return 'high'
data['policy_csl_level'] = data['policy_csl'].apply(get_csl_level)


data = pd.get_dummies(data, columns=['policy_csl_level'])


# 将'customer_months'列的值取负
data['policy_deductable_miunus'] = -data['policy_deductable']


# 将学历按照程度排序,分配不同的分数
edu_level_map = {'High School': 1, 'Associate': 2, 'College': 3,
                 'JD': 4, 'Masters': 5, 'MD': 6, 'PhD': 7}
data['edu_level_score'] = data['insured_education_level'].map(edu_level_map)


# 对职业按照收入水平赋予不同分数
occ_level_map = {'other-service': 1, 'priv-house-serv': 1, 'handlers-cleaners': 2,
                'machine-op-inspct': 3, 'farming-fishing': 3, 'transport-moving': 4,
                'craft-repair': 4, 'sales': 5, 'armed-forces': 5,
                'tech-support': 6, 'protective-serv': 6, 'adm-clerical': 7,
                'exec-managerial': 8, 'prof-specialty': 9}
data['occ_level_score'] = data['insured_occupation'].map(occ_level_map)


# 对兴趣爱好按照程度赋予不同分数
hobby_level_map = {'sleeping': 1, 'movies': 2, 'video-games': 2, 'reading': 3,
                   'exercise': 4, 'hiking': 4, 'cross-fit': 4, 'camping': 4,
                   'dancing': 5, 'golf': 6, 'chess': 6, 'board-games': 6,
                   'paintball': 7, 'kayaking': 7, 'bungie-jumping': 8,
                   'skydiving': 8, 'base-jumping': 9, 'yachting': 9, 'polo': 10}
data['hobby_level_score'] = data['insured_hobbies'].map(hobby_level_map)


# 构造综合特征
data['overall_score'] = data['edu_level_score'] + data['occ_level_score'] + data['hobby_level_score']
# 根据总分将被保人分为不同风险等级
data['risk_level'] = pd.cut(data['overall_score'], bins=[0, 10, 20, 30],
                              labels=['high', 'medium', 'low'])
data = pd.get_dummies(data, columns=['risk_level'])

data = pd.get_dummies(data, columns=['edu_level_score'])
data = pd.get_dummies(data, columns=['occ_level_score'])
data = pd.get_dummies(data, columns=['hobby_level_score'])

# 将'customer_months'列的值取负
data['capital-gains_minus'] = -data['capital-gains']

# 定义关系稳定性映射
relationship_stability_map = {'husband': 0, 'wife': 0,
                               'own-child': 1, 'other-relative': 2,
                               'not-in-family': 3, 'unmarried': 3}
# 映射关系稳定性得分
data['relationship_stability'] = data['insured_relationship'].map(relationship_stability_map)

# 构造是否稳定关系的标签
data['is_stable_relationship'] = (data['relationship_stability'] >= 3).astype(int)
data = pd.get_dummies(data, columns=['relationship_stability'])

# 对auto_make进行目标编码
target_encoder = TargetEncoder()
# data['auto_make_encoded'] = target_encoder.fit_transform(data['auto_make'], data['fraud'])
data = pd.get_dummies(data, columns=['auto_make'])


# 对auto_model进行目标编码
# data['auto_model_encoded'] = target_encoder.fit_transform(data['auto_model'], data['fraud'])
data = pd.get_dummies(data, columns=['auto_model'])

# 分箱
data['incident_hour_bin'] = pd.cut(data['incident_hour_of_the_day'], bins=[-1, 6, 12, 18, 25], labels=['night', 'morning', 'afternoon', 'evening'])
# 目标编码
# data['incident_hour_encoded'] = target_encoder.fit_transform(data['incident_hour_bin'], data['fraud'])
data = pd.get_dummies(data, columns=['incident_hour_bin'])

# 对省份和城市分别进行目标编码
# data['incident_state_encoded'] = target_encoder.fit_transform(data['incident_state'], data['fraud'])
data = pd.get_dummies(data, columns=['incident_state'])


# data['incident_city_encoded'] = target_encoder.fit_transform(data['incident_city'], data['fraud'])
data = pd.get_dummies(data, columns=['incident_city'])

# 给每个严重程度赋予一个分数
sev_map = {'Trivial Damage': 3, 'Minor Damage': 2, 'Major Damage': 1, 'Total Loss': 0}
data['incident_severity_score'] = data['incident_severity'].map(sev_map)
# data.drop(['incident_severity'], axis=1, inplace=True)
# 使用特殊值填充问号
data['collision_type'].replace('?', 'Unknown', inplace=True)
# One-hot编码
collision_dummies = pd.get_dummies(data['collision_type'], prefix='collision')
data = pd.concat([data, collision_dummies], axis=1)

# 给每个严重程度赋予一个分数
vehicle_map = {'Vehicle': 0, 'Theft': 0, 'Multi-vehicle Collision': 1, 'Single Vehicle Collision': 2}
data['incident_type'] = data['incident_type'].map(vehicle_map)
data = pd.get_dummies(data, columns=['incident_type'])

In [103]:
# 将年龄分箱
bins = [0, 18, 30, 45, 60, 120]
labels = ['child', 'young', 'adult', 'middle_aged', 'senior']
data['age_bin'] = pd.cut(data['age'], bins=bins, labels=labels)
data = pd.get_dummies(data, columns=['age_bin'])

In [104]:
# 提取policy_bind_date和incident_date的年月日
data['policy_year'] = pd.to_datetime(data['policy_bind_date']).dt.year
data['policy_month'] = pd.to_datetime(data['policy_bind_date']).dt.month
data['policy_day'] = pd.to_datetime(data['policy_bind_date']).dt.day
data['incident_year'] = pd.to_datetime(data['incident_date']).dt.year
data['incident_month'] = pd.to_datetime(data['incident_date']).dt.month
data['incident_day'] = pd.to_datetime(data['incident_date']).dt.day

# 计算policy_bind_date到incident_date的时间差
data['days_to_incident'] = (pd.to_datetime(data['incident_date']) - pd.to_datetime(data['policy_bind_date'])).dt.days

In [105]:
# # 对类别特征进行One-Hot编码
# cat_cols = ['policy_state', 'insured_sex', 'insured_education_level', 'insured_occupation', 
#             'insured_hobbies', 'insured_relationship', 'incident_type', 'collision_type',
#             'authorities_contacted', 'incident_state', 'incident_city', 'auto_make', 'auto_model','incident_severity']
# data = pd.get_dummies(data, columns=cat_cols)

In [106]:
# 计算incident_hour_of_the_day的sin和cos值
data['incident_hour_sin'] = np.sin(2 * np.pi * data['incident_hour_of_the_day'] / 24)
data['incident_hour_cos'] = np.cos(2 * np.pi * data['incident_hour_of_the_day'] / 24)

# 计算资本收益和损失的比例
data['capital_gain_loss_ratio'] = data['capital-gains'] / (data['capital-loss'] + 1)


# 填充缺失值
# data['police_report_available'] = data['police_report_available'].fillna(data['police_report_available'].mode()[0])
# data['property_damage'] = data['property_damage'].fillna(data['property_damage'].mode()[0])

In [107]:
# 计算索赔金额的总和和比例
data['total_claim_ratio'] = data['total_claim_amount'] / (data['injury_claim'] + data['property_claim'] + data['vehicle_claim'] + 1)
data['claim_sum'] = data['injury_claim'] + data['property_claim'] + data['vehicle_claim']

# 对于高度偏态的数值特征进行对数转换
skewed_features = ['capital-gains', 'capital-loss', 'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim']
for feat in skewed_features:
    data[feat] = np.log1p(data[feat])

d:\Anaconda_env\pytorch3\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [108]:
# 处理日期特征
data['policy_bind_date'] = data['policy_bind_date'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
data['incident_date'] = data['incident_date'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
data['auto_year'] = data['auto_year'].apply(lambda x: int(pd.to_datetime(x).timestamp()))

In [109]:
# data['property_damage'] = data['property_damage'].replace('?', pd.NA)
# data['police_report_available'] = data['police_report_available'].replace('?', pd.NA)

In [110]:
# data['property_damage'] = data['property_damage'].fillna(0)
# data['police_report_available'] = data['police_report_available'].fillna(0)

In [111]:
# 替换'property_damage'列的值
data['property_damage'].replace({'YES': 1, 'NO': 0, '?': 0.5}, inplace=True)
# 替换'police_report_available'列的值
data['police_report_available'].replace({'YES': 0, 'NO': 1, '?': 0.5}, inplace=True)
# data.drop(['bodily_injuries','witnesses','number_of_vehicles_involved','incident_date'], axis=1, inplace=True)
# 分箱
data['incident_hour_bin'] = pd.cut(data['incident_hour_of_the_day'], bins=[-1, 6, 12, 18, 25], labels=['night', 'morning', 'afternoon', 'evening'])

data = pd.get_dummies(data, columns=['incident_hour_bin'])


In [112]:
# # 定义一个函数来转换布尔型字符串为整数
# def str_to_int(val):
#     if val == 'YES':
#         return 1
#     elif val == 'NO':  
#         return -1
#     else:
#         return val

# # 遍历每一列,如果列中包含'yes'或'no',则应用转换函数
# for column in data.columns:
#     if data[column].dtypes == 'object':
#         data[column] = data[column].apply(str_to_int)

In [113]:
# 获取所有布尔列
bool_cols = data.dtypes[data.dtypes == bool].index.tolist()

# 遍历布尔列并替换
for col in bool_cols:
    data[col] = data[col].astype(int)
    data[col] = data[col].replace({0: -1})

In [117]:
data.drop(['policy_csl','insured_education_level','insured_occupation','insured_hobbies','insured_relationship','collision_type',	'incident_severity',	'authorities_contacted'
], axis=1, inplace=True)


In [118]:
from sklearn.preprocessing import StandardScaler
# 创建一个 StandardScaler 对象
scaler = StandardScaler()
# 对数据表中的所有列进行标准化
data_standardized = scaler.fit_transform(data)
# 将标准化后的值添加为新的列
data_standardized_df = pd.DataFrame(data_standardized, columns=[f'{col}_standardized' for col in data.columns])
# 将原始数据表和标准化后的数据表合并
data = pd.concat([data, data_standardized_df], axis=1)

In [119]:
# 对所有列进行对数转换
data_log = np.log1p(data)

# 将对数转换后的列添加到原始数据中
for col in data.columns:
    try:
        data[f'{col}_log'] = data_log[col]
    except ValueError:
        print(f"Warning: Skipping column '{col}' due to ValueError")
        continue

d:\Anaconda_env\pytorch3\lib\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: divide by zero encountered in log1p
  result = func(self.values, **kwargs)
d:\Anaconda_env\pytorch3\lib\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)
C:\Users\13410\AppData\Local\Temp\ipykernel_25672\510007158.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{col}_log'] = data_log[col]
C:\Users\13410\AppData\Local\Temp\ipykernel_25672\510007158.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inste

C:\Users\13410\AppData\Local\Temp\ipykernel_25672\510007158.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{col}_log'] = data_log[col]
C:\Users\13410\AppData\Local\Temp\ipykernel_25672\510007158.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{col}_log'] = data_log[col]
C:\Users\13410\AppData\Local\Temp\ipykernel_25672\510007158.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all column

In [ ]:
# # 获取当前列名
# columns = data.columns

# # 创建新的列名列表，使用数字作为标签
# new_columns = [str(i) for i in range(1, len(columns) + 1)]

# # 将列名改为数字标签
# data.rename(columns=dict(zip(columns, new_columns)), inplace=True)


In [ ]:
# import numpy as np
# import pandas as pd

# # 对数据表中的所有列进行对数转换
# data_log_transformed = np.log1p(data)

# data_log_transformed_df = pd.DataFrame(data_log_transformed, columns=[f'log_transformed_{col}' for col in data_log_transformed.columns])

# data = pd.concat([data, data_log_transformed_df], axis=1)

# # 重置索引标签
# data.reset_index(drop=True, inplace=True)
# data_log_transformed_df.reset_index(drop=True, inplace=True)

# # 将原始数据表和对数转换后的数据表合并
# data = pd.concat([data, data_log_transformed_df], axis=1)


In [ ]:
# from openfe import openfe, transform
# ofe = openfe()
# features =    ofe.fit(data=data, label=y, n_jobs=-1)  # generate new features
# train_x, test_x = transform(train_x, test_x, features, n_jobs=n_jobs) # transform the train and test data according to generated features.

In [120]:
from sklearn.linear_model import Lasso, LassoCV
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel

In [121]:

data = data.round(6)
# 替换正无穷大为一个较大的有限值,如1e10
data[data==np.inf] = 1e10  
# 替换负无穷大为一个较小的有限值,如-1e10
data[data==-np.inf] = -1e10

In [122]:
# 分离特征和目标变量
# 填充X中的NaN值
imputer = SimpleImputer(strategy='mean')
data = imputer.fit_transform(data)

d:\Anaconda_env\pytorch3\lib\site-packages\sklearn\impute\_base.py:577: UserWarning: Skipping features without any observed values: ['policy_deductable_miunus_log']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [123]:
y = data_target

In [ ]:
# 创建一个默认的列名列表
# default_column_names = [str(i) for i in range(1, data.shape[1] + 1)]
# lasso = LassoCV(cv=5, random_state=42, alphas=[0.001, 0.01, 0.1, 1, 10, 100])
# lasso.fit(data, y)
# # 获取特征选择器
# feature_mask = SelectFromModel(lasso, prefit=True)
# # 获取被选择的特征列的索引
# selected_features_index = feature_mask.get_support()
# # 获取被选择的特征列的列名
# selected_column_names = [default_column_names[i] for i, selected in enumerate(selected_features_index) if selected]
# # 输出被选择的特征列的列名
# print(selected_column_names)

# data = feature_mask.transform(data)

In [ ]:
# 特征选择
# from sklearn.feature_selection import f_classif, mutual_info_classif,VarianceThreshold

In [125]:
data = pd.DataFrame(data)

In [126]:
# Create a list of numerical column names
numerical_columns = [col for col in data.columns if data[col].dtype in ['int', 'float', 'np.float64']]

# Select numerical data into a new DataFrame
numerical_data = data[numerical_columns]


In [127]:
# Create and fit the PCA model
pca = PCA(n_components=3)  # Reduce to 2 dimensions
X_pca = pca.fit_transform(numerical_data)

In [ ]:
# 绘制三维散点图
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
colors = ['b', 'r']
ax.scatter(X_pca[y==0, 0], X_pca[y==0, 1], X_pca[y==0, 2], c=colors[0], label='Non-fraud')
ax.scatter(X_pca[y==1, 0], X_pca[y==1, 1], X_pca[y==1, 2], c=colors[1], label='Fraud')
ax.legend()
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2') 
ax.set_zlabel('Component 3')
plt.show()

In [128]:
X_pca = pd.DataFrame(X_pca)
# data = data.iloc[:, 5:]
data = pd.concat([data, pd.DataFrame(X_pca)], axis=1)

In [ ]:
from xgboost import XGBClassifier
import lightgbm as lgb

# XGBoost
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
feature_importances_xgb = xgb_model.feature_importances_
feature_importance_df_xgb = pd.DataFrame({'feature': X_train.columns, 'importance': feature_importances_xgb})
feature_importance_df_xgb = feature_importance_df_xgb.sort_values('importance', ascending=False)

# 选择重要性前20%的特征作为强特征
num_top_features = int(0.2 * len(X_train.columns))
top_features_xgb = feature_importance_df_xgb.head(num_top_features)['feature'].tolist()

# LightGBM
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(X_train, y_train)
feature_importances_lgb = lgb_model.feature_importances_
feature_importance_df_lgb = pd.DataFrame({'feature': X_train.columns, 'importance': feature_importances_lgb})
feature_importance_df_lgb = feature_importance_df_lgb.sort_values('importance', ascending=False)

# 选择重要性前20%的特征作为强特征
num_top_features = int(0.2 * len(X_train.columns))
top_features_lgb = feature_importance_df_lgb.head(num_top_features)['feature'].tolist()

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

# 获取特征重要性表中排名前几的数值特征
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)
top_numeric_features = [col for col in feature_importance_df.head(10)['feature'] if X_train[col].dtypes != 'object']

# 对数值特征进行交叉组合
new_features = []
for degree in range(2, 4):  # 设置多项式的最高次数为3
    for combo in combinations(top_numeric_features, degree):
        new_feature = np.prod(X_train[list(combo)], axis=1)
        new_feature_name = '_'.join(combo)
        new_features.append(new_feature_name)
        # X_train[new_feature_name] = new_feature
        # X_test[new_feature_name] = np.prod(X_test[list(combo)], axis=1)

# 查看新特征
# print(f"原始特征数量: {X_train.shape[1]}")
# print(f"新增特征数量: {len(new_features)}")
# print(f"总特征数量: {X_train.shape[1]}")

In [ ]:
selected_data_lgb = data[top_features_lgb]
selected_data_xgb = data[top_features_xgb]
data_selected = pd.concat([data_filtered, pd.DataFrame(top_features_lgb)], axis=1)
data_selected = pd.concat([data_filtered, pd.DataFrame(top_features_xgb)], axis=1)
data_selected = pd.concat([data_filtered, pd.DataFrame(new_features)], axis=1)

In [229]:
# from sklearn.feature_selection import f_classif, mutual_info_classif, VarianceThreshold

# data = pd.DataFrame(data)

# # 使用 f_classif 进行单变量特征选择
f_scores, p_values = f_classif(data, y)
selected_features_f = data.columns[(f_scores > 15) & (p_values < 0.01)]  
X_train_f = data[selected_features_f]
# # X_test_f = X_test[selected_features_f]

# # 使用 mutual_info_classif 进行互信息特征选择
mi_scores = mutual_info_classif(data, y)
selected_features_mi = data.columns[mi_scores > 0.1]  # 选择互信息大于 0.1 的特征
X_train_mi = data[selected_features_mi]
# X_test_mi = X_test[selected_features_mi]

# # 使用 VarianceThreshold 进行低方差特征过滤
# selector = VarianceThreshold(threshold=100)
# X_train_var = selector.fit_transform(data)
# X_test_var = selector.transform(X_test)

In [230]:
X_train_mi.shape

(700, 48)

In [211]:
data_filtered = pd.DataFrame()

In [212]:
data_filtered = pd.concat([data_filtered, pd.DataFrame(X_train_f)], axis=1)
data_filtered = pd.concat([data_filtered, pd.DataFrame(X_train_mi)], axis=1)
# data_filtered = pd.concat([data_filtered, pd.DataFrame(X_train_var)], axis=1)

In [213]:
from sklearn.linear_model import Lasso, LassoCV
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel

In [218]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

rf = RandomForestClassifier(random_state=42)
rf.fit(data, y)

model = SelectFromModel(rf, prefit=True, threshold=0.005)
data_rf = model.transform(data)

In [220]:
lasso = LassoCV(cv=5, random_state=42, alphas=[0.001, 0.01, 0.1, 1, 10, 100])
lasso.fit(data, y)
feature_mask = SelectFromModel(lasso, prefit=True)
data_lasso = feature_mask.transform(data)

d:\Anaconda_env\pytorch3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.774e+01, tolerance: 1.084e-02
  model = cd_fast.enet_coordinate_descent(
d:\Anaconda_env\pytorch3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.891e+01, tolerance: 1.084e-02
  model = cd_fast.enet_coordinate_descent(
d:\Anaconda_env\pytorch3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.852e+01, toleranc

In [221]:
data_filtered = pd.concat([data_filtered, pd.DataFrame(data_rf)], axis=1)
data_filtered = pd.concat([data_filtered, pd.DataFrame(data_lasso)], axis=1)

In [ ]:
# from sklearn.decomposition import PCA

# 假设 numerical_features 是数值型特征的列名列表
# pca = PCA(n_components=5)  # 选择保留的主成分数量
# principal_components = pca.fit_transform(data)
# data = pd.DataFrame(principal_components)
# data = data.iloc[:, 5:]
# data = pd.concat([data, pd.DataFrame(principal_components)], axis=1)

In [223]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

In [224]:
# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(data_filtered, y, test_size=0.2, random_state=42)

# 定义模型
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier()
}

# 定义参数搜索空间
param_grid = {
    'SVM': [{'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}],
    'Random Forest': [{'n_estimators': [2, 4, 6, 8, 10]}]
}

# 定义评估指标
scoring = ['f1', 'precision', 'recall', 'roc_auc']

results = []
for name, model in models.items():
    if name in param_grid:
        grid = GridSearchCV(model, param_grid[name], scoring=scoring, cv=5, refit='f1')
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        print(f'{name} best params: {grid.best_params_}')
    else:
        best_model = model.fit(X_train, y_train)
    
    y_pred = best_model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    
    results.append([name, f1, precision, recall, auc])

# 打印结果
print('Model, F1, Precision, Recall, AUC')
for result in results:
    print(', '.join([str(x) for x in result]))

d:\Anaconda_env\pytorch3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Anaconda_env\pytorch3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Anaconda_env\pytorch3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted 

SVM best params: {'C': 0.001, 'gamma': 0.001}
Random Forest best params: {'n_estimators': 10}
Model, F1, Precision, Recall, AUC
Logistic Regression, 0.047619047619047616, 0.14285714285714285, 0.02857142857142857, 0.48571428571428565
Decision Tree, 0.36363636363636365, 0.3333333333333333, 0.4, 0.5666666666666667
KNN, 0.24615384615384617, 0.26666666666666666, 0.22857142857142856, 0.5095238095238096
SVM, 0.0, 0.0, 0.0, 0.5
Random Forest, 0.46153846153846156, 0.5, 0.42857142857142855, 0.6428571428571429


In [234]:
# 导入必要的库
from sklearn.model_selection import KFold
from shap import KernelExplainer, summary_plot

# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(data_filtered, y, test_size=0.2, random_state=42)

# 定义模型
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(C= 0.001, gamma= 0.001),  # 使用上面提取出来的最佳参数
    'Random Forest': RandomForestClassifier(n_estimators= 10)  # 使用上面提取出来的最佳参数
}

# 定义评估指标
scoring = ['f1', 'precision', 'recall', 'roc_auc']

# 5-折交叉验证
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results = []
for name, model in models.items():
    cv_scores = []
    for train_idx, val_idx in kf.split(X_train):
        X_train_cv, X_val_cv = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_cv, y_val_cv = y_train.iloc[train_idx], y_train.iloc[val_idx]
        model.fit(X_train_cv, y_train_cv)
        y_pred_cv = model.predict(X_val_cv)
        f1 = f1_score(y_val_cv, y_pred_cv)
        precision = precision_score(y_val_cv, y_pred_cv)
        recall = recall_score(y_val_cv, y_pred_cv)
        auc = roc_auc_score(y_val_cv, y_pred_cv)
        cv_scores.append([f1, precision, recall, auc])
    avg_scores = np.mean(cv_scores, axis=0)
    results.append([name] + list(avg_scores))

# 打印结果
print('Model, F1, Precision, Recall, AUC')
for result in results:
    print(', '.join([str(x) for x in result]))

# 绘制SHAP值图
explainer = KernelExplainer(model.predict, X_train)
shap_values = explainer.shap_values(X_train)
summary_plot(shap_values, X_train, plot_type="bar")

d:\Anaconda_env\pytorch3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Anaconda_env\pytorch3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Model, F1, Precision, Recall, AUC
Logistic Regression, 0.18146927793986617, 0.4546031746031747, 0.11652941176470588, 0.5296620770841817
Decision Tree, 0.3512021737833814, 0.3483285386700021, 0.3653872549019608, 0.562351603341084
KNN, 0.2746250207400033, 0.41847826086956524, 0.21219117647058824, 0.5489937627409118
SVM, 0.0, 0.0, 0.0, 0.5
Random Forest, 0.47379448878892705, 0.5482600732600733, 0.422985294117647, 0.6495907849035971


  0%|          | 0/560 [00:00<?, ?it/s]

In [225]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

In [226]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=2)
tpot.fit(data_filtered, y)
print(tpot.score(data_filtered, y))

Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]


2.00 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: DecisionTreeClassifier(input_matrix, criterion=entropy, max_depth=1, min_samples_leaf=2, min_samples_split=13)
0.7985714285714286


In [154]:
# 获取每个样本属于每个类别的概率预测值
y_proba = tpot.predict(data_filtered)

# 计算ROC曲线下的面积（AUC）
auc = roc_auc_score(y, y_proba)
print("AUC:", auc)

AUC: 0.7474105536571605


In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


In [ ]:
# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(data_filtered, y, test_size=0.2, random_state=42,shuffle=False)

# 定义模型和参数搜索空间
models = {
    'LightGBM': (lgb.LGBMClassifier(random_state=42), {'num_leaves': [10, 30, 50, 70, 80], 'max_depth': [5, 10, 15, 30, 50]}),
    'XGBoost': (XGBClassifier(random_state=42), {'max_depth': [5, 10, 15, 20, 50], 'n_estimators': [80, 100, 200, 300]}),
    'CatBoost': (CatBoostClassifier(random_state=42), {'max_depth': [3, 5, 7, 10], 'n_estimators': [100, 200, 300, 500]})
}


# 定义评估指标
scoring = ['f1', 'precision', 'recall', 'roc_auc']

# 使用GridSearchCV选择最佳模型和参数
best_model = None
best_score = 0
best_params = None

for name, (model, param_grid) in models.items():
    grid = GridSearchCV(model, param_grid, scoring='f1', cv=5, refit=True)
    grid.fit(data, y)
    score = grid.best_score_
    if score > best_score:
        best_model = grid.best_estimator_
        best_score = score
        best_params = grid.best_params_
        print(f'{name} best params: {best_params}, best score: {best_score}')

# 在测试集上评估最佳模型
y_pred = best_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

# 打印结果
print(f'Best Model: {best_model.__class__.__name__}')
print(f'F1: {f1}, Precision: {precision}, Recall: {recall}, AUC: {auc}')

In [ ]:
# 读取数据
data2 = pd.read_csv('test.csv')
# data = data.drop(data.columns[-1], axis=1)

# 与平均年龄的差值
age_mean = data2['age'].mean()
data2['age_diff_mean'] = np.abs(data2['age'] - age_mean)

# 将'customer_months'列的值取负
data2['customer_months_miunus'] = -data2['customer_months']


# 将'policy_state'和'insured_sex'列转换为独热编码
data2 = pd.get_dummies(data2, columns=['policy_state', 'insured_sex'])
# 构造policy_csl_total特征
def get_csl_total(csl_str):
    csl_min, csl_max = map(int, csl_str.split('/'))
    return csl_min + csl_max
data2['policy_csl_total'] = data2['policy_csl'].apply(get_csl_total)


# 构造policy_csl_level特征
def get_csl_level(csl_str):
    csl_min, csl_max = map(int, csl_str.split('/'))
    csl_total = csl_min + csl_max
    if csl_total < 500:
        return 'low'
    elif csl_total < 1000:
        return 'medium'
    else:
        return 'high'
data2['policy_csl_level'] = data2['policy_csl'].apply(get_csl_level)


data2 = pd.get_dummies(data2, columns=['policy_csl_level'])


# 将'customer_months'列的值取负
data2['policy_deductable_miunus'] = -data2['policy_deductable']


# 将学历按照程度排序,分配不同的分数
edu_level_map = {'High School': 1, 'Associate': 2, 'College': 3,
                 'JD': 4, 'Masters': 5, 'MD': 6, 'PhD': 7}
data2['edu_level_score'] = data2['insured_education_level'].map(edu_level_map)


# 对职业按照收入水平赋予不同分数
occ_level_map = {'other-service': 1, 'priv-house-serv': 1, 'handlers-cleaners': 2,
                'machine-op-inspct': 3, 'farming-fishing': 3, 'transport-moving': 4,
                'craft-repair': 4, 'sales': 5, 'armed-forces': 5,
                'tech-support': 6, 'protective-serv': 6, 'adm-clerical': 7,
                'exec-managerial': 8, 'prof-specialty': 9}
data2['occ_level_score'] = data2['insured_occupation'].map(occ_level_map)


# 对兴趣爱好按照程度赋予不同分数
hobby_level_map = {'sleeping': 1, 'movies': 2, 'video-games': 2, 'reading': 3,
                   'exercise': 4, 'hiking': 4, 'cross-fit': 4, 'camping': 4,
                   'dancing': 5, 'golf': 6, 'chess': 6, 'board-games': 6,
                   'paintball': 7, 'kayaking': 7, 'bungie-jumping': 8,
                   'skydiving': 8, 'base-jumping': 9, 'yachting': 9, 'polo': 10}
data2['hobby_level_score'] = data2['insured_hobbies'].map(hobby_level_map)


# 构造综合特征
data2['overall_score'] = data2['edu_level_score'] + data2['occ_level_score'] + data2['hobby_level_score']
# 根据总分将被保人分为不同风险等级
data2['risk_level'] = pd.cut(data2['overall_score'], bins=[0, 10, 20, 30],
                              labels=['high', 'medium', 'low'])
data2 = pd.get_dummies(data2, columns=['risk_level'])

data2 = pd.get_dummies(data2, columns=['edu_level_score'])
data2 = pd.get_dummies(data2, columns=['occ_level_score'])
data2 = pd.get_dummies(data2, columns=['hobby_level_score'])

# 将'customer_months'列的值取负
data2['capital-gains_minus'] = -data2['capital-gains']

# 定义关系稳定性映射
relationship_stability_map = {'husband': 0, 'wife': 0,
                               'own-child': 1, 'other-relative': 2,
                               'not-in-family': 3, 'unmarried': 3}
# 映射关系稳定性得分
data2['relationship_stability'] = data2['insured_relationship'].map(relationship_stability_map)

# 构造是否稳定关系的标签
data2['is_stable_relationship'] = (data2['relationship_stability'] >= 3).astype(int)
data2 = pd.get_dummies(data2, columns=['relationship_stability'])

# 对auto_make进行目标编码

data2 = pd.get_dummies(data2, columns=['auto_make'])


# 对auto_model进行目标编码

data2 = pd.get_dummies(data2, columns=['auto_model'])

# 分箱
data2['incident_hour_bin'] = pd.cut(data2['incident_hour_of_the_day'], bins=[-1, 6, 12, 18, 25], labels=['night', 'morning', 'afternoon', 'evening'])
# 目标编码
data2 = pd.get_dummies(data2, columns=['incident_hour_bin'])

# 对省份和城市分别进行目标编码

data2 = pd.get_dummies(data2, columns=['incident_state'])



data2 = pd.get_dummies(data2, columns=['incident_city'])

# 给每个严重程度赋予一个分数
sev_map = {'Trivial Damage': 3, 'Minor Damage': 2, 'Major Damage': 1, 'Total Loss': 0}
data2['incident_severity_score'] = data2['incident_severity'].map(sev_map)
# data.drop(['incident_severity'], axis=1, inplace=True)
# 使用特殊值填充问号
data2['collision_type'].replace('?', 'Unknown', inplace=True)
# One-hot编码
collision_dummies = pd.get_dummies(data2['collision_type'], prefix='collision')
data2 = pd.concat([data2, collision_dummies], axis=1)

# 给每个严重程度赋予一个分数
vehicle_map = {'Vehicle': 0, 'Theft': 0, 'Multi-vehicle Collision': 1, 'Single Vehicle Collision': 2}
data2['incident_type'] = data2['incident_type'].map(vehicle_map)
data2 = pd.get_dummies(data2, columns=['incident_type'])

# 将年龄分箱
bins = [0, 18, 30, 45, 60, 120]
labels = ['child', 'young', 'adult', 'middle_aged', 'senior']
data2['age_bin'] = pd.cut(data2['age'], bins=bins, labels=labels)
data2 = pd.get_dummies(data2, columns=['age_bin'])

# 提取policy_bind_date和incident_date的年月日
data2['policy_year'] = pd.to_datetime(data2['policy_bind_date']).dt.year
data2['policy_month'] = pd.to_datetime(data2['policy_bind_date']).dt.month
data2['policy_day'] = pd.to_datetime(data2['policy_bind_date']).dt.day
data2['incident_year'] = pd.to_datetime(data2['incident_date']).dt.year
data2['incident_month'] = pd.to_datetime(data2['incident_date']).dt.month
data2['incident_day'] = pd.to_datetime(data2['incident_date']).dt.day

# 计算policy_bind_date到incident_date的时间差
data2['days_to_incident'] = (pd.to_datetime(data2['incident_date']) - pd.to_datetime(data2['policy_bind_date'])).dt.days

# 计算incident_hour_of_the_day的sin和cos值
data2['incident_hour_sin'] = np.sin(2 * np.pi * data2['incident_hour_of_the_day'] / 24)
data2['incident_hour_cos'] = np.cos(2 * np.pi * data2['incident_hour_of_the_day'] / 24)

# 计算资本收益和损失的比例
data2['capital_gain_loss_ratio'] = data2['capital-gains'] / (data2['capital-loss'] + 1)


# 填充缺失值
# data['police_report_available'] = data['police_report_available'].fillna(data['police_report_available'].mode()[0])
# data['property_damage'] = data['property_damage'].fillna(data['property_damage'].mode()[0])

# 计算索赔金额的总和和比例
data2['total_claim_ratio'] = data2['total_claim_amount'] / (data2['injury_claim'] + data2['property_claim'] + data2['vehicle_claim'] + 1)
data2['claim_sum'] = data2['injury_claim'] + data2['property_claim'] + data2['vehicle_claim']

# 对于高度偏态的数值特征进行对数转换
skewed_features = ['capital-gains', 'capital-loss', 'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim']
for feat in skewed_features:
    data2[feat] = np.log1p(data2[feat])

    # 处理日期特征
data2['policy_bind_date'] = data2['policy_bind_date'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
data2['incident_date'] = data2['incident_date'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
data2['auto_year'] = data2['auto_year'].apply(lambda x: int(pd.to_datetime(x).timestamp()))

# 替换'property_damage'列的值
data2['property_damage'].replace({'YES': 1, 'NO': 0, '?': 0.5}, inplace=True)
# 替换'police_report_available'列的值
data2['police_report_available'].replace({'YES': 0, 'NO': 1, '?': 0.5}, inplace=True)
# data.drop(['bodily_injuries','witnesses','number_of_vehicles_involved','incident_date'], axis=1, inplace=True)
# 分箱
data2['incident_hour_bin'] = pd.cut(data2['incident_hour_of_the_day'], bins=[-1, 6, 12, 18, 25], labels=['night', 'morning', 'afternoon', 'evening'])

data2 = pd.get_dummies(data2, columns=['incident_hour_bin'])


# 获取所有布尔列
bool_cols = data2.dtypes[data2.dtypes == bool].index.tolist()

# 遍历布尔列并替换
for col in bool_cols:
    data2[col] = data2[col].astype(int)
    data2[col] = data2[col].replace({0: -1})

data2.drop(['policy_csl','insured_education_level','insured_occupation','insured_hobbies','insured_relationship','collision_type',	'incident_severity',	'authorities_contacted'], axis=1, inplace=True)

from sklearn.preprocessing import StandardScaler
# 创建一个 StandardScaler 对象
scaler = StandardScaler()
# 对数据表中的所有列进行标准化
data_standardized = scaler.fit_transform(data2)
# 将标准化后的值添加为新的列
data_standardized_df = pd.DataFrame(data_standardized, columns=[f'{col}_standardized' for col in data2.columns])
# 将原始数据表和标准化后的数据表合并
data2 = pd.concat([data2, data_standardized_df], axis=1)

# 对所有列进行对数转换
data_log = np.log1p(data2)

# 将对数转换后的列添加到原始数据中
for col in data2.columns:
    try:
        data2[f'{col}_log'] = data_log[col]
    except ValueError:
        print(f"Warning: Skipping column '{col}' due to ValueError")
        continue


data2 = data2.round(6)
# 替换正无穷大为一个较大的有限值,如1e10
data2[data2==np.inf] = 1e10  
# 替换负无穷大为一个较小的有限值,如-1e10
data2[data2==-np.inf] = -1e10

# 分离特征和目标变量
# 填充X中的NaN值
imputer = SimpleImputer(strategy='mean')
data2 = imputer.fit_transform(data2)


In [ ]:
data2 = pd.DataFrame(data2)
data2.columns = [str(i) for i in range(1, len(data2.columns) + 1)]
selected_columns = ['3', '5', '6', '12', '16', '19', '20', '23', '156', '158', '159', '183', '184', '186', '295', '311', '345', '506', '563', '636', '649', '658']
data2 = data2[selected_columns]


# 基于方差选择特征
selector = VarianceThreshold()
data2 = selector.fit_transform(data2)

In [ ]:
data2 = pd.DataFrame(data2)
data2

In [ ]:
# 获取每个样本属于每个类别的概率预测值
y_proba_test = tpot.predict(data2)

In [ ]:
y_proba_test

In [ ]:
y_pre_test = best_model.predict(data2)

In [ ]:
y_pre_test = pd.DataFrame(y_pre_test)

In [ ]:
y_pre_test

In [ ]:
y_pre_test = y_pre_test 1241

测试集

In [ ]:
# 读取数据
data_test = pd.read_csv('第五周/test.csv')
# 将年龄分箱
bins = [0, 18, 30, 45, 60, 120]
labels = ['child', 'young', 'adult', 'middle_aged', 'senior']
data_test['age_bin'] = pd.cut(data_test['age'], bins=bins, labels=labels)
# 提取policy_bind_date和incident_date的年月日
data_test['policy_year'] = pd.to_datetime(data_test['policy_bind_date']).dt.year
data_test['policy_month'] = pd.to_datetime(data_test['policy_bind_date']).dt.month
data_test['policy_day'] = pd.to_datetime(data_test['policy_bind_date']).dt.day
data_test['incident_year'] = pd.to_datetime(data_test['incident_date']).dt.year
data_test['incident_month'] = pd.to_datetime(data_test['incident_date']).dt.month
data_test['incident_day'] = pd.to_datetime(data_test['incident_date']).dt.day

In [ ]:
# 计算policy_bind_date到incident_date的时间差
data_test['days_to_incident'] = (pd.to_datetime(data_test['incident_date']) - pd.to_datetime(data_test['policy_bind_date'])).dt.days
# 对类别特征进行One-Hot编码
cat_cols = ['policy_state', 'insured_sex', 'insured_education_level', 'insured_occupation', 
            'insured_hobbies', 'insured_relationship', 'incident_type', 'collision_type',
            'authorities_contacted', 'incident_state', 'incident_city', 'auto_make', 'auto_model','incident_severity']
data_test = pd.get_dummies(data_test, columns=cat_cols)
# 计算incident_hour_of_the_day的sin和cos值
data_test['incident_hour_sin'] = np.sin(2 * np.pi * data_test['incident_hour_of_the_day'] / 24)
data_test['incident_hour_cos'] = np.cos(2 * np.pi * data_test['incident_hour_of_the_day'] / 24)

# 计算资本收益和损失的比例
data_test['capital_gain_loss_ratio'] = data_test['capital-gains'] / (data_test['capital-loss'] + 1)

# 填充缺失值
data_test['police_report_available'] = data_test['police_report_available'].fillna(data_test['police_report_available'].mode()[0])
data_test['property_damage'] = data_test['property_damage'].fillna(data_test['property_damage'].mode()[0])
# 计算索赔金额的总和和比例
data_test['total_claim_ratio'] = data_test['total_claim_amount'] / (data_test['injury_claim'] + data_test['property_claim'] + data_test['vehicle_claim'] + 1)
data_test['claim_sum'] = data_test['injury_claim'] + data_test['property_claim'] + data_test['vehicle_claim']

# 对于高度偏态的数值特征进行对数转换
skewed_features = ['capital-gains', 'capital-loss', 'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim']
for feat in skewed_features:
    data_test[feat] = np.log1p(data_test[feat])
    # 处理日期特征
data_test['policy_bind_date'] = data_test['policy_bind_date'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
data_test['incident_date'] = data_test['incident_date'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
data_test['auto_year'] = data_test['auto_year'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
data_test['property_damage'] = data_test['property_damage'].replace('?', pd.NA)
data_test['police_report_available'] = data_test['police_report_available'].replace('?', pd.NA)
data_test['property_damage'] = data_test['property_damage'].fillna(0)
data_test['police_report_available'] = data_test['police_report_available'].fillna(0)

In [ ]:
# 定义一个函数来转换布尔型字符串为整数
def str_to_int(val):
    if val == 'YES':
        return 1
    elif val == 'NO':
        return -1
    else:
        return val

# 遍历每一列，如果列中包含'yes'或'no'，则应用转换函数
for column in data_test.columns:
    if data_test[column].dtype == 'object':
        data_test[column] = data_test[column].apply(str_to_int)
        # 迭代遍历所有列
for column in data_test.columns:
    # 如果列的数据类型是布尔类型
    if data_test[column].dtype == bool:
        # 将布尔类型的列转换为整数类型，并将 True 转换为 1，False 转换为 -1
        data_test[column] = data_test[column].astype(int)
        data_test[column] = data_test[column].replace({0: -1})

# 定义一个函数来转换布尔型字符串为整数
def str_to_int1(val):
    if val == 'YES':
        return 1
    elif val == 'NO':
        return -1
    else:
        return val

# 遍历每一列，如果列中包含'yes'或'no'，则应用转换函数
for column in data_test.columns:
    if data_test[column].dtype == 'object':
        data_test[column] = data_test[column].apply(str_to_int1)
# 定义一个函数来转换布尔型字符串为整数

def str_to_int2(val):
    if val == 'TRUE':
        return 1
    elif val == 'FALSE':
        return -1
    else:
        return val

# 遍历每一列，如果列中包含'yes'或'no'，则应用转换函数
for column in data_test.columns:
    if data_test[column].dtype == 'object':
        data_test[column] = data_test[column].apply(str_to_int2)
        
data_test= data_test.replace('?', pd.NA)

In [ ]:
data_test = data_test.drop(columns=['policy_id','age_bin','policy_csl'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

# 最小-最大标准化
minmax_scaler = MinMaxScaler()
data_test = minmax_scaler.fit_transform(data_test)

In [ ]:
data_test.columns = [str(i) for i in range(1, len(data_test.columns) + 1)]
selected_columns = ['1', '2', '4', '5', '8', '11', '13', '15', '19', '23', '24', '26', '27', '30', '32', '34', '44', '59', '60', '61', '75', '78', '87', '88', '89', '91', '94', '95', '131', '157', '160']
data_test = data_test[selected_columns]

In [ ]:
# 基于方差选择特征
selector = VarianceThreshold()
data_test = selector.fit_transform(data_test)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler, Normalizer

# # 最小-最大标准化
# minmax_scaler = MinMaxScaler()
# data_test = minmax_scaler.fit_transform(data_test)

# # 单位向量标准化
# normalizer = Normalizer(norm='l2')  # l2 表示使用欧几里得范数
# data_test = normalizer.fit_transform(data_test)

# z-score标准化
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# data_test = scaler.fit_transform(data_test)

In [ ]:
# from sklearn.decomposition import PCA

# # 假设 numerical_features 是数值型特征的列名列表
# pca = PCA(n_components=5)  # 选择保留的主成分数量
# principal_components = pca.fit_transform(data_test)
# data_test = pd.DataFrame(data_test)
# data_test = data_test.iloc[:, 5:]
# data_test = pd.concat([data_test, pd.DataFrame(principal_components)], axis=1)

In [ ]:
data_test

In [ ]:
data_test.shape

In [ ]:
y_pred_test = best_model.predict(data_test)
y_pred_test


In [ ]:
# 定义一个函数来转换布尔型字符串为整数
def str_to_int(val):
    if val == 'YES':
        return 1
    elif val == 'NO':
        return 0
    else:
        return val

# 遍历每一列，如果列中包含'yes'或'no'，则应用转换函数
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = data[column].apply(str_to_int)

In [ ]:
data['property_damage'] = data['property_damage'].replace('?', pd.NA)
data['police_report_available'] = data['police_report_available'].replace('?', pd.NA)

In [ ]:
data['property_damage'] = data['property_damage'].fillna(0.5)
data['police_report_available'] = data['police_report_available'].fillna(0.5)

In [ ]:
# # 处理日期特征
# data['policy_bind_date'] = data['policy_bind_date'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
# data['incident_date'] = data['incident_date'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
# data['auto_year'] = data['auto_year'].apply(lambda x: int(pd.to_datetime(x).timestamp()))


In [ ]:
# 处理类别特征
categorical_cols = ['policy_state', 'insured_sex', 'insured_education_level', 'insured_occupation', 'insured_hobbies', 'insured_relationship', 'incident_type', 'collision_type', 'authorities_contacted', 'incident_state', 'incident_city', 'auto_make', 'auto_model']
encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    encoders[col] = le

# 处理One-Hot编码
ohe = OneHotEncoder(sparse=False)
categorical_data = data[categorical_cols]
categorical_data = ohe.fit_transform(categorical_data)

In [ ]:
categorical_data

In [ ]:
# 处理数值特征
numerical_cols = ['age', 'customer_months', 'policy_deductable', 'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'capital-gains', 'capital-loss', 'incident_hour_of_the_day', 'number_of_vehicles_involved', 'bodily_injuries', 'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim','incident_date', 'policy_id']
scaler = StandardScaler()
numerical_data = scaler.fit_transform(data[numerical_cols])

# 合并编码后的特征
X = np.concatenate([categorical_data, numerical_data,data[['police_report_available', 'property_damage']]], axis=1)
y = data['fraud']

In [ ]:
X

In [ ]:
# 使用PCA降维到3维
pca = PCA(n_components=7)
X_pca = pca.fit_transform(X)

# 绘制三维散点图
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
colors = ['b', 'r']
ax.scatter(X_pca[y==0, 0], X_pca[y==0, 1], X_pca[y==0, 2], c=colors[0], label='Non-fraud')
ax.scatter(X_pca[y==1, 0], X_pca[y==1, 1], X_pca[y==1, 2], c=colors[1], label='Fraud')
ax.legend()
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2') 
ax.set_zlabel('Component 3')
plt.show()

In [ ]:
X_pca

In [ ]:
# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定义模型
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier()
}

# 定义参数搜索空间
param_grid = {
    'SVM': [{'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}],
    'Random Forest': [{'n_estimators': [2, 4, 6, 8, 10]}]
}

# 定义评估指标
scoring = ['f1', 'precision', 'recall', 'roc_auc']

results = []
for name, model in models.items():
    if name in param_grid:
        grid = GridSearchCV(model, param_grid[name], scoring=scoring, cv=5, refit='f1')
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        print(f'{name} best params: {grid.best_params_}')
    else:
        best_model = model.fit(X_train, y_train)
    
    y_pred = best_model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    
    results.append([name, f1, precision, recall, auc])

# 打印结果
print('Model, F1, Precision, Recall, AUC')
for result in results:
    print(', '.join([str(x) for x in result]))

In [ ]:
# 导入必要的库
from sklearn.model_selection import KFold
from shap import KernelExplainer, summary_plot

# 定义模型
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(**grid.best_params_['SVM']),  # 使用上面提取出来的最佳参数
    'Random Forest': RandomForestClassifier(**grid.best_params_['Random Forest'])  # 使用上面提取出来的最佳参数
}

# 定义评估指标
scoring = ['f1', 'precision', 'recall', 'roc_auc']

# 5-折交叉验证
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results = []
for name, model in models.items():
    cv_scores = []
    for train_idx, val_idx in kf.split(X_train):
        X_train_cv, X_val_cv = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_cv, y_val_cv = y_train.iloc[train_idx], y_train.iloc[val_idx]
        model.fit(X_train_cv, y_train_cv)
        y_pred_cv = model.predict(X_val_cv)
        f1 = f1_score(y_val_cv, y_pred_cv)
        precision = precision_score(y_val_cv, y_pred_cv)
        recall = recall_score(y_val_cv, y_pred_cv)
        auc = roc_auc_score(y_val_cv, y_pred_cv)
        cv_scores.append([f1, precision, recall, auc])
    avg_scores = np.mean(cv_scores, axis=0)
    results.append([name] + list(avg_scores))

# 打印结果
print('Model, F1, Precision, Recall, AUC')
for result in results:
    print(', '.join([str(x) for x in result]))

# 绘制SHAP值图
explainer = KernelExplainer(model.predict, X_train)
shap_values = explainer.shap_values(X_train)
summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
# 导入必要的库
import lightgbm as lgb
from xgboost import XGBClassifier


In [ ]:
# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定义模型和参数搜索空间
models = {
    'LightGBM': (lgb.LGBMClassifier(random_state=42), {'num_leaves': [30, 50, 70,100], 'max_depth': [5, 10, 15,20,50]}),
    'XGBoost': (XGBClassifier(random_state=42), {'max_depth': [5, 10, 15,20,50], 'n_estimators': [30,50,80]}),
}

# 定义评估指标
scoring = ['f1', 'precision', 'recall', 'roc_auc']

# 使用GridSearchCV选择最佳模型和参数
best_model = None
best_score = 0
best_params = None

for name, (model, param_grid) in models.items():
    grid = GridSearchCV(model, param_grid, scoring='f1', cv=5, refit=True)
    grid.fit(X_train, y_train)
    score = grid.best_score_
    if score > best_score:
        best_model = grid.best_estimator_
        best_score = score
        best_params = grid.best_params_
        print(f'{name} best params: {best_params}, best score: {best_score}')

# 在测试集上评估最佳模型
y_pred = best_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

# 打印结果
print(f'Best Model: {best_model.__class__.__name__}')
print(f'F1: {f1}, Precision: {precision}, Recall: {recall}, AUC: {auc}')

In [ ]:
y_pred

In [ ]:
y_pred_test